# Downloads and Imports

In [ ]:
!pip install --quiet requests openai jsonschema

In [ ]:
import json
from openai import OpenAI
from google.colab import userdata
from typing import List

# Configurations, Model Instances and Constans

In [ ]:
CHAT_MODEL = "llama-3.3-70b-versatile"
STRUCTURED_OUTPUT_MODEL = "openai/gpt-oss-20b"

In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key = userdata.get("GROQ_API_KEY")
)

In [48]:
MESSAGE_HISTORY : List[dict] = []

In [49]:
summarization_interval = 8
turn_count = 0

In [50]:
SUMMARY = ""

In [51]:
CHAT_MODEL_SYSTEM_PROMPT = """

"""

# Conversation Manager Utilities

## Function to add messages

In [56]:
def add_message(message):
    global turn_count
    role = message['role']
    MESSAGE_HISTORY.append(message)
    turn_count += 1
    print(f"Turn {turn_count}: Added {message}")

    if turn_count % summarization_interval == 0:
      print(f"Turn count {turn_count}, output: {turn_count % summarization_interval}")
      periodic_summarization()

## functionm to create summary

In [59]:
def generate_summary(messages):
    global SUMMARY
    global turn_count
    global MESSAGE_HISTORY

    if not MESSAGE_HISTORY:
        print("No messages to summarize.")
        return
    conversation_text = "\n".join([f"{message['role']}: {message['content']}" for message in MESSAGE_HISTORY])

    summary_prompt = f"""
        Please provide a concise summary of the following conversation, in the third person point of view. Focus on:
        1. Main topics discussed
        2. Key decisions or outcomes
        3. Important information exchanged
        4. Overall context and flow

        Conversation:
        {conversation_text}

        Summary:"""
    try:
        chat_bot_response = client.chat.completions.create(
          model=CHAT_MODEL,
            messages=[{"role":"user", "content":conversation_text}],
            max_tokens=150,
            temperature=0.3)
        summary = chat_bot_response.choices[0].message.content.strip()
        print(f"✅ Generated summary ({len(summary)} characters)")
        return summary
    except Exception as e:
        print(f"❌ Error generating summary: {e}")
        return "Summary generation failed"

## Function to create periodic summary

In [60]:
def periodic_summarization():
        global MESSAGE_HISTORY
        global SUMMARY

        # Only generate a new summary if there's enough new conversation
        if len(MESSAGE_HISTORY) >= 2:
            current_summary = generate_summary(MESSAGE_HISTORY)

            if SUMMARY:
                # Combine with previous summary
                combined_prompt = f"""
                Previous summary: {SUMMARY}

                New conversation summary: {current_summary}

                Please create a comprehensive summary that combines both:"""

                try:
                    response = client.chat.completions.create(
                        model=CHAT_MODEL,
                        messages=[{"role": "user", "content": combined_prompt}],
                        max_tokens= 300,
                        temperature=0.3
                    )

                    SUMMARY = response.choices[0].message.content.strip()

                except Exception as e:
                    print(f"❌ Error combining summaries: {e}")
                    SUMMARY = current_summary
            else:
                SUMMARY = current_summary

            # Replace history with summary
            MESSAGE_HISTORY = [
                {"role": "system", "content": CHAT_MODEL_SYSTEM_PROMPT},
                {"role": "system", "content": f"Previous conversation summary: {SUMMARY}"}
            ]

            print(f"Conversation summarized and history reset")
            print(f"Current summary length: {len(SUMMARY)} characters")

# Test Bot

In [57]:
while True:
    user_input = input("User: ")
    add_message({"role": "user", "content": user_input})

    try:
        chat_bot_response = client.chat.completions.create(
            model=CHAT_MODEL,
            messages=MESSAGE_HISTORY
        )
        response_content = chat_bot_response.choices[0].message.content
        add_message({"role": "assistant", "content": response_content})

        print(f"Bot: {response_content}")

    except Exception as e:
        print(f"An error occurred: {e}")

User: Hi, how are you?
Turn 8: Added {'role': 'user', 'content': 'Hi, how are you?'}
Turn 9: Added {'role': 'assistant', 'content': "Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or topics you'd like to discuss. How about you? How's your day going so far? Is there something specific you'd like to talk about, or would you like to continue exploring the history of computers and civilizations like we started discussing earlier?"}
Bot: Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or topics you'd like to discuss. How about you? How's your day going so far? Is there something specific you'd like to talk about, or would you like to continue exploring the history of computers and civilizations like we started di

KeyboardInterrupt: Interrupted by user

In [58]:
SUMMARY

'It seems like you\'re interested in learning more about the history of computers, civilizations, and the most expensive computers and chips in the world. I\'d be happy to help you explore these topics further.\n\nTo recap, we discussed the following:\n\n**History of Computers and Civilizations:**\n\n1. **The first computer**: We talked about the different contenders for the title of "first computer," including Charles Babbage\'s Analytical Engine, Konrad Zuse\'s Z1, Colossus, and ENIAC. These early machines paved the way for the development of modern computers.\n2. **The oldest civilization**: We explored some of the oldest known civilizations, including Sumer, Egypt, and Mesopotamia, and discussed their contributions to the development of writing, governance, and technology.\n\n**Most Expensive Computers and Chips:**\n\n1. **The most expensive computer**: We talked about the Summit Supercomputer, which is currently the world\'s fastest supercomputer, with a price tag of'